In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


def read_files(base_dir):
    all_data = []
    for subject_folder in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_folder)
        if os.path.isdir(subject_path):
            subject_number = int(subject_folder.split("_")[-1])
            for csv_file in os.listdir(subject_path):
                if csv_file.endswith("arm_r.csv"):
                    file_path = os.path.join(subject_path, csv_file)
                    df = pd.read_csv(file_path)
                    df["subject"] = subject_number
                    df["body_part"] = csv_file.split(".")[0]
                    all_data.append(df)
    return pd.concat(all_data, ignore_index=True)


def normalize_features(data):
    scaler = StandardScaler()
    columns_to_normalize = ["ax", "ay", "az", "wx", "wy", "wz", "ex", "ey", "ez"]
    data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
    return data


def encode_labels(data):
    label_encoding = {
        "fall": 0, "run": 1, "walk": 2, "cycle": 3, "lay": 4, "squat": 5, "mop": 6, "drink": 7, "sweep": 8,
        "brushing_teeth": 9, "cut": 10, "eat": 11, "folding_clothes": 12, "hang_out_clothes": 13, "ironing": 14,
        "open_door": 15, "open_fridge": 16, "sit": 17, "stand": 18, "use_computer": 19, "wash_dish": 20,
        "wash_face": 21, "wash_window": 22, "watch_tv": 23, "watering_flowers": 24, "write": 25, "wc": 26,
        "play_phone": 27, "switch": 28
    }
    data["encoded_label"] = data["label"].map(label_encoding)
    return data


def apply_sliding_window(data, window_size=100, step_size=50):
    windows = []
    labels = []
    for subject in data["subject"].unique():
        subject_data = data[data["subject"] == subject].sort_values("time")
        for i in range(0, len(subject_data) - window_size + 1, step_size):
            window = subject_data.iloc[i:i + window_size]
            windows.append(window[["ax", "ay", "az", "wx", "wy", "wz", "ex", "ey", "ez"]].values.flatten())
            labels.append(window["encoded_label"].mode().values[0])  # Most common label in the window
    return np.array(windows), np.array(labels)


def prepare_datasets(data):
    subjects = sorted(data["subject"].unique())
    train_subjects = subjects[:25]
    test_subjects = subjects[25:]

    train_data = data[data["subject"].isin(train_subjects)]
    test_data = data[data["subject"].isin(test_subjects)]

    X_train, y_train = apply_sliding_window(train_data)
    X_test, y_test = apply_sliding_window(test_data)

    return X_train, y_train, X_test, y_test


def save_datasets(X_train, y_train, X_test, y_test):
    np.savetxt("X_train.txt", X_train)
    np.savetxt("y_train.txt", y_train, fmt="%d")
    np.savetxt("X_test.txt", X_test)
    np.savetxt("y_test.txt", y_test, fmt="%d")


def worker(base_dir):
    # Read and combine all data
    data = read_files(base_dir)

    # Normalize features
    data = normalize_features(data)

    # Encode labels
    data = encode_labels(data)

    # Prepare datasets
    X_train, y_train, X_test, y_test = prepare_datasets(data)

    # Save datasets
    save_datasets(X_train, y_train, X_test, y_test)

    print("DatasetPreprocessing completed. Datasets saved as txt files.")


base_dir = "../../../../Datasets/CAPP Dataset/data"
worker(base_dir)

Preprocessing completed. Datasets saved as txt files.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt

# Load the preprocessed data
X_train = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_train.txt")
y_train = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_train.txt")
X_test = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

# Reshape the input data for Conv1D layers
n_timesteps = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timesteps, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timesteps, n_features)

# One-hot encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)


def create_model(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(n_timesteps, n_features)),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation="relu"),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation="relu"),
        BatchNormalization(),
        LSTM(100, return_sequences=True),
        Dropout(0.5),
        LSTM(100),
        Dropout(0.5),
        Dense(100, activation="relu"),
        BatchNormalization(),
        Dense(n_outputs, activation="softmax")
    ])
    return model


n_outputs = y_train.shape[1]
model = create_model(n_timesteps, n_features, n_outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

2024-09-26 09:15:44.983427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 09:15:45.038121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 09:15:45.067084: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 09:15:45.163523: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 09:15:45.866380: W tensorflow/compiler/tf2

Epoch 1/50


2024-09-26 09:16:11.603823: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1727320571.728481   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.758848   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.759385   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.759932   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.769777   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.776826   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.778963   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320571.791768   62135 gpu_t

   1/2495 ━━━━━━━━━━━━━━━━━━━━ 2:31:09 4s/step - accuracy: 0.0312 - loss: 3.7235

W0000 00:00:1727320573.112666   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.113736   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.114269   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.114870   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.115941   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.116629   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.117157   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.117809   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320573.120246   62135 gp

2495/2495 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4712 - loss: 1.7030

W0000 00:00:1727320604.355116   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.355695   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.356207   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.356711   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.357226   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.357751   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.358264   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.358782   62133 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320604.359300   62133 gp

2495/2495 ━━━━━━━━━━━━━━━━━━━━ 38s 14ms/step - accuracy: 0.4712 - loss: 1.7029 - val_accuracy: 0.7066 - val_loss: 1.0308
Epoch 2/50
   6/2495 ━━━━━━━━━━━━━━━━━━━━ 26s 11ms/step - accuracy: 0.7234 - loss: 0.9090 

W0000 00:00:1727320607.262575   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.263171   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.263683   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.264192   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.264698   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.265197   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.265687   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.266205   62135 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727320607.266706   62135 gp

2495/2495 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - accuracy: 0.7068 - loss: 0.9496 - val_accuracy: 0.7283 - val_loss: 0.9484
Epoch 3/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step - accuracy: 0.7721 - loss: 0.7447 - val_accuracy: 0.7518 - val_loss: 0.8949
Epoch 4/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step - accuracy: 0.8096 - loss: 0.6242 - val_accuracy: 0.7624 - val_loss: 0.8566
Epoch 5/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - accuracy: 0.8275 - loss: 0.5609 - val_accuracy: 0.7556 - val_loss: 0.8379
Epoch 6/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step - accuracy: 0.8445 - loss: 0.5081 - val_accuracy: 0.7859 - val_loss: 0.8152
Epoch 7/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - accuracy: 0.8611 - loss: 0.4507 - val_accuracy: 0.7666 - val_loss: 0.8976
Epoch 8/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - accuracy: 0.8754 - loss: 0.4089 - val_accuracy: 0.7871 - val_loss: 0.8149
Epoch 9/50
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - accuracy: 0.8791 - loss: 0.38

In [3]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, digits=5))
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(122)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()

616/616 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    recall  f1-score   support

           0    0.91601   0.88242   0.89890       927
           1    0.98361   0.97087   0.97720       927
           2    0.90841   0.91234   0.91037       924
           3    0.96575   0.80388   0.87741       877
           4    0.88068   0.81924   0.84885       946
           5    0.69979   0.71915   0.70934       940
           6    0.92157   0.94597   0.93361       944
           7    0.86566   0.84105   0.85318       950
           8    0.93926   0.45531   0.61331       951
           9    0.76376   0.35922   0.48863       927
          10    0.86355   0.71331   0.78128      1322
          11    0.59166   0.95567   0.73085       564
          12    0.91718   0.96141   0.93878       933
          13    0.66750   0.85546   0.74988       934
          14    0.72378   0.82234   0.76992       940
          15    0.67181   0.84483   0.74845       928
          16    0.96967   0.99688   0.98

In [4]:
from everywhereml.code_generators.tensorflow import convert_model

c_header = convert_model(model, X_test, y_test, model_name='deep_conv_lstm')
print(c_header)

with open('GeneratedHeaderFiles/deep_conv_lstm.h', 'w') as file:
    file.write(c_header)


INFO:tensorflow:Assets written to: /tmp/tmp46knjafe/assets


INFO:tensorflow:Assets written to: /tmp/tmp46knjafe/assets


Saved artifact at '/tmp/tmp46knjafe'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  136028022062272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020198384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020202432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020203136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020202080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020204368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020206832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020209120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020204720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020209648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136028020205952: T

W0000 00:00:1727322279.956060   60885 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727322279.956074   60885 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-26 09:44:39.956293: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp46knjafe
2024-09-26 09:44:39.957184: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-26 09:44:39.957194: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp46knjafe
2024-09-26 09:44:39.966577: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-26 09:44:39.968400: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-26 09:44:40.022411: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp46knjafe
2024-09-26 09:44:40.040162: I tensorflow/cc/saved_model/loader.cc

ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_1/lstm_1/CudnnRNNV3@__inference_function_657136"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_657275"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.CudnnRNNV3' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_1/lstm_1/CudnnRNNV3@__inference_function_657136"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_657275"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): Error code: ERROR_NEEDS_CUSTOM_OPS
<unknown>:0: error: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_1/lstm_1_2/CudnnRNNV3@__inference_function_657136"] at fused["StatefulPartitionedCall:",